##  Lazily Computed Property

1. **Purpose of Lazily Computed Property:**
   - Lazily computed properties are used to improve performance by avoiding unnecessary calculations until the property is actually needed.

2. **Use of Descriptors:**
   - Descriptors in Python are objects that define how attributes are accessed, set, or deleted in a class. The `lazyproperty` class is acting as a descriptor.

3. **Descriptor Triggering Methods:**
   - In general, when a descriptor is used in a class, its `__get__`, `__set__`, and `__delete__` methods are triggered every time the property is accessed, set, or deleted.



### Example

The `lazyproperty` decorator allows you to create properties that are computed only once and then cached for future use. Here's a breakdown of the code:


1. **`lazyproperty` Class:**
   - This is a custom class acting as a descriptor.
   - It takes a function (`func`) during initialization, which is assumed to be a method of the class that will be decorated.

2. **`__init__` Method:**
   - The `__init__` method stores the function passed to it (the method to be lazily evaluated).

3. **`__get__` Method:**
   - The `__get__` method is called when the decorated property is accessed.
   - The `if instance is None:` check is necessary because the `__get__` method can also be called when the property is being accessed on the class itself, rather than on an instance of the class. In this case, the instance parameter will be None.
      - If accessed on the class itself (`instance is None`), it returns the `lazyproperty` instance.
      - If accessed on an instance, it computes the value using the stored function and sets the attribute on the instance with the computed value. It then returns the computed value.

4. **Usage in the `Circle` Class:**
   - The `Circle` class has two properties: `area` and `perimeter`, both decorated with `@lazyproperty`.
   - The `area` and `perimeter` properties are computed lazily, meaning that the calculations are performed only when the properties are first accessed, and the results are cached for subsequent accesses.

5. **Example Usage:**
   - Accessing the `area` and `perimeter` properties triggers the lazy computation, and the results are printed.
   - Subsequent accesses to the properties reuse the previously computed values without recalculating.

This approach is a form of memoization, where the results of expensive function calls are cached to avoid redundant computations. It's particularly useful for properties or methods that involve heavy calculations but might not always be needed.

In [1]:
class lazyproperty:
    def __init__(self, func) -> None:
        self.func = func

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value

In [2]:
import math

class Circle:
    def __init__(self, radius) -> None:
        self.radius = radius

    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2

    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius

In [3]:
circle_instance = Circle(radius=5)

In [4]:
circle_instance.area

Computing area


78.53981633974483

In [5]:
# Accessing the area propety again reuses the previously computed value. No computation
circle_instance.area

78.53981633974483

In [6]:
circle_instance.perimeter

Computing perimeter


31.41592653589793

In [7]:
# Accessing the perimeter propety again reuses the previously computed value. No computation
circle_instance.perimeter

31.41592653589793